<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/model-blending/blend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging CodeLLMs to Create an Efficant Low-Memory Quantized Model for `whats-up-doc`

## Download and Install `mergekit`

In [1]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 2265, done.
remote: Counting objects: 100% (1338/1338), done.
remote: Compressing objects: 100% (517/517), done.
remote: Total 2265 (delta 1065), reused 935 (delta 820), pack-reused 927
Receiving objects: 100% (2265/2265), 640.43 KiB | 2.43 MiB/s, done.
Resolving deltas: 100% (1583/1583), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 31.6 MB/s 

## Create the YAML Config File to Merge Models with SLERP

In [2]:
import os
import yaml
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline

In [3]:
!pip install huggingface-cli

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credenti

### Write Config Script

In [5]:
# Set the resultant model's name
MODEL_NAME = 'whats-up-llamas'

MODEL_1 = "codellama/CodeLlama-7b-Instruct-hf"
MODEL_2 = "meta-llama/Meta-Llama-3-8B-Instruct"

OUTPUT_DIR = "merged_model"

#### SLERP

In [17]:
yamlConfigSLERPLlamas = f"""
slices:
  - sources:
      - model: {MODEL_1}
        layer_range: [0, 32]
      - model: {MODEL_2}
        layer_range: [0, 32]
merge_method: slerp
base_model: {MODEL_1}
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: float32
"""

#### Passthrough

In [7]:
yamlConfigPassthrough = """
slices:
  - sources:
    - model: OpenPipe/mistral-ft-optimized-1218
      layer_range: [0, 32]
  - sources:
    - model: mlabonne/NeuralHermes-2.5-Mistral-7B
      layer_range: [24, 32]
merge_method: passthrough
dtype: bfloat16

"""

*Note: If you were to do this locally, instead of putting in the models' card name under `model`, you would specify the path to the model you downloaded from huggingface.*

#### DARE-TIES

In [26]:
yamlConfigDARETIESLlamas = f"""
models:
    # No parameters necessary for base model
  - model: {MODEL_1}
  - model: {MODEL_2}
    parameters:
      density: 0.53
      weight: 0.4
merge_method: dare_ties
base_model: {MODEL_1}
parameters:
  int8_mask: true
dtype: bfloat16

"""

### Save Config Script

In [27]:
# Save the YAML configuration to a file
yamlFileName = "config.yaml"
with open(yamlFileName, "w") as f:
    f.write(yamlConfigDARETIESLlamas)

## Merge Models

In [28]:
cmd = f"mergekit-yaml {yamlFileName} {OUTPUT_DIR} --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle"
!{cmd}

Warmup loader cache:   0% 0/2 [00:00<?, ?it/s]
Fetching 10 files: 100% 10/10 [00:00<00:00, 10787.82it/s]
Warmup loader cache:  50% 1/2 [00:00<00:00,  1.61it/s]
Fetching 11 files: 100% 11/11 [00:00<00:00, 30076.50it/s]
Warmup loader cache: 100% 2/2 [00:01<00:00,  1.61it/s]
Executing graph:   0% 3/1457 [00:00<06:24,  3.78it/s]WARNING:root:Using submatrix of meta-llama/Meta-Llama-3-8B-Instruct:lm_head.weight
Executing graph:   1% 8/1457 [00:10<29:15,  1.21s/it]WARNING:root:Using submatrix of meta-llama/Meta-Llama-3-8B-Instruct:model.embed_tokens.weight
Executing graph:   1% 10/1457 [00:20<1:08:39,  2.85s/it]WARNING:root:skipping meta-llama/Meta-Llama-3-8B-Instruct:model.layers.0.mlp.down_proj.weight due to size mismatch
Executing graph:   2% 24/1457 [00:21<13:43,  1.74it/s]WARNING:root:skipping meta-llama/Meta-Llama-3-8B-Instruct:model.layers.0.mlp.gate_proj.weight due to size mismatch
Executing graph:   2% 28/1457 [00:22<10:57,  2.17it/s]WARNING:root:skipping meta-llama/Meta-Llama-3-8B-I